##COPY and MERGE DBX

Loads data from a file location into a Delta table. This is a retryable and idempotent operation — Files in the source location that have already been loaded are skipped. This is true even if the files have been modified since they were loaded.

In [0]:
%fs
mkdirs dbfs:/Volumes/gizmobox/landing/operational_data/stock_price

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gizmobox.`delta-lake`.raw_stock_prices

In [0]:
%sql
select * from json.`/Volumes/gizmobox/landing/operational_data/stock_price/`

In [0]:
%sql
COPY INTO gizmobox.`delta-lake`.raw_stock_prices 
FROM '/Volumes/gizmobox/landing/operational_data/stock_price/'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true')

In [0]:
%sql
SELECT * FROM gizmobox.`delta-lake`.raw_stock_prices 

THIS SHOULD CREATE 5 RECORDS

In [0]:
%sql
SELECT * FROM gizmobox.`delta-lake`.raw_stock_prices 

In [0]:
%sql
CREATE TABLE gizmobox.`delta-lake`.stock_prices (
  stock_id STRING,
  price DOUBLE,
  trading_date DATE,
  status STRING
)
--USING DELTA
---LOCATION '/path/to/delta-lake/stock_prices';

In [0]:
%sql
MERGE INTO gizmobox.`delta-lake`.stock_prices AS target
USING gizmobox.`delta-lake`.raw_stock_prices AS source
ON target.stock_id = source.stock_id
WHEN MATCHED AND source.status = 'ACTIVE' THEN
  UPDATE SET target.price = source.price, target.trading_date = source.trading_date
WHEN MATCHED AND source.status = 'DELISTED' THEN
  DELETE
WHEN NOT MATCHED AND source.status = 'ACTIVE' THEN
  INSERT (stock_id, price, trading_date) VALUES (source.stock_id, source.price, source.trading_date);

In [0]:
%sql select * from gizmobox.`delta-lake`.stock_prices

In [0]:
%sql
select * from gizmobox.`delta-lake`.raw_stock_prices 

In [0]:
%sql
--DELETE from gizmobox.`delta-lake`.raw_stock_prices;
COPY INTO gizmobox.`delta-lake`.raw_stock_prices 
FROM '/Volumes/gizmobox/landing/operational_data/stock_price/'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

In [0]:
%sql
COPY INTO gizmobox.`delta-lake`.raw_stock_prices 
FROM '/Volumes/gizmobox/landing/operational_data/stock_price/'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true')